In [4]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
sample = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [ ]:
validation = train[29400:]
train = train[:29400]

In [ ]:
train_data = train.drop('label', axis =1).to_numpy()
train_images = train_data.reshape(-1, 28, 28, 1)
train_images = train_images.astype('float32') / 255.0

In [ ]:
validation_data = validation.drop('label', axis =1).to_numpy()
validation_labels = validation['label']

validation_data_scaled = validation_data / 255.0
    
# Reshape validation_data to (batch_size, height, width, channels)
validation_data_reshaped = validation_data_scaled.reshape(-1, 28, 28, 1)

# Construct the validation data tuple
validation_data_tuple = (validation_data_reshaped, validation_labels)

In [ ]:
train_images.shape, validation_data.shape

In [ ]:
# Define the zoom range
zoom_range = (0.8, 1.2)  # Zoom in/out by a factor between 0.8 and 1.2

train_labels = train['label']
# Define the ImageDataGenerator with zoom augmentation options
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    zoom_range=zoom_range  # Zoom in/out on images by the specified range
)

# Fit the ImageDataGenerator on your training data
train_datagen.fit(train_images)  # Assuming train_data is your training dataset containing images


# Generate augmented data batches
batch_size = 32
augmented_train = train_datagen.flow(train_images, train_labels,  batch_size=batch_size)

# Now you can use the augmented_train_data for training your model


In [ ]:
import tensorflow as tf

# Define your model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model 
model.fit(augmented_train, validation_data = validation_data_tuple, epochs = 50)

# Prediction

In [ ]:
test_data = test.to_numpy()

test_data_scaled = test_data / 255.0
    
# Reshape validation_data to (batch_size, height, width, channels)
test_data_reshaped = test_data_scaled.reshape(-1, 28, 28, 1)

In [ ]:
test_data_reshaped.shape

In [ ]:
y_preds = model.predict(test_data_reshaped)

In [ ]:
final = y_preds.argmax(axis =1)

In [ ]:
image_id = np.arange(1,28001)
image_id

In [ ]:
submission = pd.DataFrame({'ImageId': image_id, 'Label': final})
submission.to_csv('submission.csv', index = False)

In [ ]:
submission

In [ ]:
tf.keras.utils.plot_model(model= model)